In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Polygon Meshes
The polygon mesh is the mainstay of three-dimensional computer graphics. What raster images are to two-dimensional graphics, meshes are to the representation of boundary surfaces that underlie many three-dimensional modeling and rendering applications.

Meshes are discrete in nature, as they describe the surface boundary of a three-dimensional form using a collection of joined polygonal faces.

The Decod.es Mesh offers only minimal functionality.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P28.jpg" style="width: 200px; display: inline;">


## Elements of a Mesh

There are three basic elements of a polygonal mesh: 
* The mesh ***vertices*** that each describe a coordinate location that lies on the approximated surface
* The mesh ***edges*** that each relate a pair of vertices that may be connected by a line segment
* The mesh ***faces***, each of which is a region that approximates a patch of the surface and that is bounded by a closed loop of edges

A number of mature approaches to the representation of meshes have been developed. Most of these separate the description of ***mesh geometry***, the coordinates represented by the vertices, from ***mesh connectivity***, the relationships between vertices that give rise to edges and faces. The differences between these approaches primarily concern how connectivity is represented. Each strikes a balance between speed of access and processing on the one hand, and concise descriptions that result in smaller file sizes on the other.



<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P12.jpg" style="width: 600px; display: inline;">


Let’s examine a range of possible implementations in a simple format. Using the
mesh shown in the nearby diagram, the tables on the following page demonstrate the storage schema for three commonly-used meshes: the ***vertex-vertex*** mesh, the ***face-vertex*** mesh, and the ***winged-edge*** mesh. While not exhaustive of mesh formats in practice, these three offer a sampling of the common approaches, from the compact to the feature-rich.

#### Vertex-Vertex Mesh

This simplest representation of a mesh stores a collection of vertices, each of which refers to the other vertices in the collection to which it is connected. The resulting data structure is compact, but faces and edges are implicit, and therefore must be calculated on demand.

TODO: Table

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P13.jpg" style="width: 200px; display: inline;">


#### Face-Vertex Mesh

In addition to the vertex list of the vertex-vertex mesh, this format stores a collection of faces that relates vertices to one another in groups of three or four. This is ***the most widely applied representation*** for meshes, and is the one adopted by Decod.es Mesh objects.

TODO: Table

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P14.jpg" style="width: 200px; display: inline;">


#### Winged-Edge Mesh

The winged-edge mesh and its variants are the most ornate and the most recently developed formats for representing a mesh. Here, explicit and redundant relationships are defined in three tables. The first two, a vertex list and face list, are identical to those described above. The third table lists edges, and relates these to the vertices which define them, the faces that they bound, and the other edges that participate in the bounding of these faces. A variant of the winged-edge mesh, the half-edge, has gained popularity recently, and is valued for the flexibility it provides for dynamically altering mesh geometry.

TODO: Table

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P15.jpg" style="width: 200px; display: inline;">


## Mesh Objects in Decod.es

The Decod.es library offers an exceedingly simple implementation of a polygon mesh. Like a PLine, PGon, and RGon, a Decod.es Mesh is a special kind of HasPts, inheriting the `hazbasis.basis` coordinate system, and the `hazpts._verts` vertices collection along with all the functions responsible for managing the vertices.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D68 Mesh Large.jpg" style="width: 800px; display: inline;">

A single member is added to the parent class members. 

The `msh._faces` member represents individual mesh faces as references to the vertices that comprise them. ***Each mesh face is described as a collection of indices***: Integers that allow us to retrieve a specific set of Point objects from the `msh._verts` collection.

In [ ]:
"""
Mesh Initialization
Meshes are constructed in much the same way as other HasPts objects, relying on 
arguments passed through to the superclass constructor for the defining of basic 
members, and only then initializing a private collection to store faces.
"""
class Mesh(HasPts):
    def __init__(self, vertices=None, faces=None, basis=None):
         #HasPts constructor handles initalization of verts and basis
        super(Mesh,self).__init__(vertices,basis)
        self._faces = [] if (faces is None) else faces

While a Mesh must allow manipulation of its `msh._verts` and `msh._faces` collections, it does so only in a managed way. 

Even in this spare face-vertex style mesh, the relationships defined by these members are interdependent, and must be kept in close synchronization. There are many considerations to take into account when providing structures for managing these relationships. Among these are:
* If a vertex is added or removed, all faces that reference it should be updated. Additionally, since vertices are referenced by index, references to higher-numbered vertices should be modified to reflect the new numbering.
* Before adding or removing faces, it should be confirmed that the action does not produce a non-manifold mesh.
* Before altering faces, it should be verified that all referenced vertices exist, and any new face shares at least one edge with an existing face.

Almost none of this is done in a Decod.es Mesh. Instead, the modest functions found below provide nearly direct access to the `msh._faces` collection, and it is left to the users of the Decod.es library to code with caution.

In [ ]:
"""
Face Management
The Decod.es Mesh offers minimal methods for managing faces. In the first method
below, a single face that relates three or four Points is added to the private 
collection _faces. The second method below defines a property faces that 
provides access to this private collection.
"""
def add_face(self,a,b,c,d=None):
    if is not None : self._faces.append([a,b,c,d])
    else: self._faces.append([a,b,c])
    
@property
def faces(self):
    return self._faces

The Decod.es Mesh does not provide the wealth of geometric properties of more elaborate mesh types. Still, a few essentials may be easily obtained.

In [ ]:
"""
Querying Face Properties
Given the minimal structure of a Decod.es Mesh, a range of geometric properties 
may be calculated for any mesh face identified by index. The first function 
below simply returns the points of the desired face. Using this information, 
the second calculates its centroid. Finally, the third calculates the normal 
vector of the desired face, accounting for four-sided faces where necessary.
"""
def face_pts(self,idx):
    return [self.pts[i] for i in self.faces[idx]]
    
def face_centroid(self,idx):
    return Point.centroid(self.face_pts(idx))
    
def face_normal(self,idx):
    pts = self.face_pts(idx)
    va = Vec(pts[0],pts[1]).cross(Vec(pts[0],pts[2])).normalized()
    if len(verts) == 3 : return va
    
    vb = Vec(pts[2],pts[3]).cross(Vec(pts[2],pts[1])).normalized()
    return Vec.bisector(va,vb).normalized()

### Mesh Usage

The sequence on subdivision demonstrated multiple ways to refine a collection of faces. These faces were represented by the data structures available to us at the time: a collection of connected Segments. We now have a data structure at our disposal that is tailor-made to handle this type of problem that involves both geometry and connectivity.

The Decod.es Mesh is more than capable to handle this refinement in a more succinct and more legible form than was possible before.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P21.jpg" style="width: 200px; display: inline;">


In [ ]:
"""
Mesh Quad Corner-to-Center Subdivision
"""
def quadsub_corner_to_ctr(msh, fac_idx):
    # append a Point at the face center, and find index
    msh.append( msh.face_centroid(fac_idx) )
    ctr_idx = len(msh)-1
    # find indices of corners of existing face
    cnr_idxs = msh._faces[fac_idx]
    
    # add four new faces
    msh.add_face(cnr_idxs[0],cnr_idxs[1],ctr_idx)
    msh.add_face(cnr_idxs[1],cnr_idxs[2],ctr_idx)
    msh.add_face(cnr_idxs[2],cnr_idxs[3],ctr_idx)
    msh.add_face(cnr_idxs[3],cnr_idxs[0],ctr_idx)
    
    # remove the existing face
    del msh._faces[fac_idx]

Notice that the ***directionality*** of each subface, that is, the counter-clockwise or clockwise order of the indices determining each subface, preserves the directionality of the original face. 

This consistency ensures that the face normals are all pointing in the same direction, and demonstrates the sort of careful bookkeeping that becomes even more important when a mesh is used to approximate a surface for which the general direction of its normals determines its orientability.